# DKW Controller Implementation - Interactive Demo

This notebook demonstrates the **DKW (Dvoretzky-Kiefer-Wolfowitz) Controller** for making fusion/fission decisions with statistical guarantees.

## Overview
The DKW Controller uses statistical bounds to decide between two operating modes:
- **Fusion**: Aggressive mode for better performance 
- **Fission**: Conservative mode for better reliability

The controller provides theoretical guarantees on error rates using the DKW inequality.

In [ ]:
# Import required libraries
import json
import numpy as np
from dataclasses import dataclass, field
import matplotlib.pyplot as plt
import pandas as pd

# Set random seed for reproducible results
np.random.seed(42)

print("✅ Libraries imported successfully!")
print("📊 Ready to run DKW Controller experiments")

## 📊 Dataset Configuration

The experimental dataset contains examples with varying difficulty levels. Each example has:
- **ID**: Unique identifier 
- **Difficulty**: Probability of error occurrence (0.0 to 1.0)

In real scenarios, this data would come from external files, but here we inline it for self-containment.

In [ ]:
# Inline dataset - replaces reading from external JSON files
# This simulates the data that would be in "../dataset_001/data_out.json"

experimental_data = [
    {"id": "example_000", "difficulty": 0.1},
    {"id": "example_001", "difficulty": 0.05}, 
    {"id": "example_002", "difficulty": 0.3},
    {"id": "example_003", "difficulty": 0.15},
    {"id": "example_004", "difficulty": 0.08},
    {"id": "example_005", "difficulty": 0.25},
    {"id": "example_006", "difficulty": 0.12},
    {"id": "example_007", "difficulty": 0.18},
    {"id": "example_008", "difficulty": 0.06},
    {"id": "example_009", "difficulty": 0.22},
    {"id": "example_010", "difficulty": 0.04},
    {"id": "example_011", "difficulty": 0.28},
    {"id": "example_012", "difficulty": 0.14},
    {"id": "example_013", "difficulty": 0.09},
    {"id": "example_014", "difficulty": 0.31},
]

print(f"📈 Dataset loaded with {len(experimental_data)} examples")
print(f"🎯 Difficulty range: {min(ex['difficulty'] for ex in experimental_data):.2f} - {max(ex['difficulty'] for ex in experimental_data):.2f}")

# Display first few examples
print("\n📋 Sample data:")
for i, ex in enumerate(experimental_data[:5]):
    print(f"  {ex['id']}: difficulty = {ex['difficulty']}")

## 🧮 DKW Controller Implementation

The DKW Controller uses the **Dvoretzky-Kiefer-Wolfowitz inequality** to provide statistical guarantees on empirical error bounds. 

### Key Parameters:
- **epsilon_target**: Target error threshold (10%)
- **delta**: Confidence level parameter (5% risk)
- **min_samples**: Minimum observations before switching modes
- **hysteresis**: Prevents oscillation between modes

### Algorithm:
1. Collect error observations over time
2. Compute DKW epsilon bound: `ε = √(ln(2/δ) / (2n))`
3. Calculate error upper bound: `empirical_error + ε`
4. Make fusion/fission decision based on bounds

In [ ]:
@dataclass
class DKWController:
    """DKW-guided fusion/fission controller with statistical guarantees."""
    
    # Configuration parameters
    epsilon_target: float = 0.10      # Target error threshold (10%)
    delta: float = 0.05              # Confidence parameter (5% risk)
    min_samples: int = 100           # Minimum samples before mode switching
    hysteresis: float = 0.05         # Hysteresis to prevent oscillation

    # State tracking
    samples: list = field(default_factory=list)
    current_state: str = "fission"   # Start in conservative mode

    def dkw_epsilon(self, n: int) -> float:
        """
        Compute DKW epsilon bound for n samples.
        
        The DKW inequality provides: P(|F_n(x) - F(x)| > ε) ≤ 2e^(-2nε²)
        Solving for ε given confidence δ: ε = √(ln(2/δ) / (2n))
        """
        if n < 2:
            return 1.0  # Conservative bound for very few samples
        return np.sqrt(np.log(2 / self.delta) / (2 * n))

    def add_observation(self, error: float) -> None:
        """Add error observation for calibration."""
        self.samples.append(error)

    def decide(self) -> str:
        """
        Make fusion/fission decision with DKW statistical guarantee.
        
        Returns:
            "fusion" for aggressive mode or "fission" for conservative mode
        """
        n = len(self.samples)
        
        # Need sufficient samples before making decisions
        if n < self.min_samples:
            return self.current_state

        # Compute DKW bound and error estimate
        epsilon = self.dkw_epsilon(n)
        empirical_error = np.mean(self.samples[-self.min_samples:])  # Use recent samples
        error_upper_bound = empirical_error + epsilon
        
        # State transition logic with hysteresis
        if self.current_state == "fusion":
            # Switch to conservative if error bound exceeds target + hysteresis
            if error_upper_bound > self.epsilon_target + self.hysteresis:
                self.current_state = "fission"
                print(f"🔄 Switching to FISSION: error_bound={error_upper_bound:.3f} > target={self.epsilon_target + self.hysteresis:.3f}")
        else:  # current_state == "fission"
            # Switch to aggressive if error bound is below target - hysteresis  
            if error_upper_bound < self.epsilon_target - self.hysteresis:
                self.current_state = "fusion"
                print(f"🔄 Switching to FUSION: error_bound={error_upper_bound:.3f} < target={self.epsilon_target - self.hysteresis:.3f}")

        return self.current_state
    
    def get_stats(self) -> dict:
        """Get current controller statistics."""
        n = len(self.samples)
        if n == 0:
            return {"samples": 0, "empirical_error": 0, "epsilon": 1.0, "upper_bound": 1.0}
            
        epsilon = self.dkw_epsilon(n)
        empirical_error = np.mean(self.samples[-self.min_samples:]) if n >= self.min_samples else np.mean(self.samples)
        upper_bound = empirical_error + epsilon
        
        return {
            "samples": n,
            "empirical_error": empirical_error,
            "epsilon": epsilon,
            "upper_bound": upper_bound,
            "current_state": self.current_state
        }

print("✅ DKWController class defined successfully!")
print("🎛️ Ready to create controller instances")

## 🧪 Experiment Runner

The experiment compares two strategies:
1. **Baseline**: Always uses conservative "fission" mode
2. **Proposed**: Uses DKW controller to adaptively choose modes

For each example, we:
- Simulate error occurrence based on difficulty 
- Feed errors to the DKW controller
- Record decisions and outcomes
- Compare performance between strategies

In [ ]:
def run_experiment(data, verbose=True):
    """
    Run DKW controller experiment comparing baseline vs proposed approach.
    
    Args:
        data: List of examples with 'id' and 'difficulty' fields
        verbose: Whether to print progress updates
        
    Returns:
        Dictionary with 'baseline' and 'proposed' results
    """
    
    controller = DKWController()
    results = {"baseline": [], "proposed": []}
    
    if verbose:
        print("🚀 Starting experiment...")
        print(f"📊 Processing {len(data)} examples...")
        print("=" * 50)
    
    for i, example in enumerate(data):
        # Simulate error occurrence based on difficulty
        # Higher difficulty = higher chance of error
        error = np.random.random() < example["difficulty"]
        
        # Feed error observation to controller
        controller.add_observation(float(error))
        
        # Get DKW controller decision
        decision = controller.decide()
        
        # Record results for proposed method
        results["proposed"].append({
            "id": example["id"],
            "decision": decision,
            "error": error,
            "difficulty": example["difficulty"]
        })
        
        # Baseline always chooses conservative "fission" mode
        results["baseline"].append({
            "id": example["id"], 
            "decision": "fission",  # Always conservative
            "error": error,
            "difficulty": example["difficulty"]
        })
        
        # Progress updates
        if verbose and i % 5 == 0:
            stats = controller.get_stats()
            print(f"📈 Step {i:2d}: {example['id']} | "
                  f"Error: {error} | Decision: {decision} | "
                  f"Samples: {stats['samples']:3d} | "
                  f"Error bound: {stats['upper_bound']:.3f}")
    
    if verbose:
        print("=" * 50)
        print("✅ Experiment completed!")
        
        # Summary statistics
        proposed_fusion_count = sum(1 for r in results["proposed"] if r["decision"] == "fusion")
        total_errors = sum(1 for r in results["proposed"] if r["error"])
        
        print(f"📊 Summary:")
        print(f"   Total examples: {len(data)}")
        print(f"   Total errors: {total_errors}")
        print(f"   Proposed fusion decisions: {proposed_fusion_count}")
        print(f"   Baseline fusion decisions: 0 (always fission)")
    
    return results

print("✅ Experiment function ready!")
print("🎯 Ready to run comparative analysis")

## 🏃‍♂️ Running the Experiment

Let's execute the experiment and see how the DKW controller performs compared to the baseline approach.

In [ ]:
# Run the experiment
results = run_experiment(experimental_data, verbose=True)

# Store results (equivalent to writing method_out.json in original script)
output_data = {
    "baseline": results["baseline"],
    "proposed": results["proposed"]
}

print(f"\n💾 Results stored with {len(results['baseline'])} baseline and {len(results['proposed'])} proposed outcomes")

## 📊 Results Analysis

Let's analyze the results and create visualizations to understand the behavior of both approaches.

In [ ]:
# Convert results to DataFrames for easier analysis
df_baseline = pd.DataFrame(results["baseline"])
df_proposed = pd.DataFrame(results["proposed"])

# Add a method column for easy comparison
df_baseline["method"] = "baseline"
df_proposed["method"] = "proposed"

# Combine for comparative analysis
df_combined = pd.concat([df_baseline, df_proposed], ignore_index=True)

print("📈 Results Summary:")
print("=" * 40)

for method in ["baseline", "proposed"]:
    method_data = df_combined[df_combined["method"] == method]
    
    total_examples = len(method_data)
    total_errors = method_data["error"].sum()
    fusion_decisions = (method_data["decision"] == "fusion").sum()
    fission_decisions = (method_data["decision"] == "fission").sum()
    
    error_rate = total_errors / total_examples if total_examples > 0 else 0
    fusion_rate = fusion_decisions / total_examples if total_examples > 0 else 0
    
    print(f"\n🔍 {method.upper()} Method:")
    print(f"   Total examples: {total_examples}")
    print(f"   Errors: {total_errors} ({error_rate:.1%})")
    print(f"   Fusion decisions: {fusion_decisions} ({fusion_rate:.1%})")
    print(f"   Fission decisions: {fission_decisions} ({(1-fusion_rate):.1%})")

# Show decision timeline
print("\n📅 Decision Timeline (Proposed Method):")
print("=" * 40)
for i, row in df_proposed.iterrows():
    decision_icon = "⚡" if row["decision"] == "fusion" else "🛡️"
    error_icon = "❌" if row["error"] else "✅"
    print(f"{decision_icon} {row['id']}: {row['decision']} | Error: {error_icon} | Difficulty: {row['difficulty']:.2f}")

In [ ]:
# Create visualizations
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle("DKW Controller vs Baseline Analysis", fontsize=16, fontweight='bold')

# 1. Decision distribution comparison
methods = ["baseline", "proposed"]
fusion_counts = [df_combined[df_combined["method"] == method]["decision"].value_counts().get("fusion", 0) for method in methods]
fission_counts = [df_combined[df_combined["method"] == method]["decision"].value_counts().get("fission", 0) for method in methods]

x = np.arange(len(methods))
width = 0.35

axes[0, 0].bar(x - width/2, fusion_counts, width, label='Fusion', color='orange', alpha=0.8)
axes[0, 0].bar(x + width/2, fission_counts, width, label='Fission', color='blue', alpha=0.8)
axes[0, 0].set_xlabel('Method')
axes[0, 0].set_ylabel('Number of Decisions')
axes[0, 0].set_title('Decision Distribution by Method')
axes[0, 0].set_xticks(x)
axes[0, 0].set_xticklabels(['Baseline', 'Proposed'])
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# 2. Error rates by difficulty
difficulty_bins = [0.0, 0.1, 0.2, 0.3, 1.0]
bin_labels = ['Low\n(0-0.1)', 'Medium\n(0.1-0.2)', 'High\n(0.2-0.3)', 'Very High\n(0.3+)']

for method, color in zip(['baseline', 'proposed'], ['blue', 'orange']):
    method_data = df_combined[df_combined["method"] == method]
    
    bin_errors = []
    for i in range(len(difficulty_bins)-1):
        mask = (method_data["difficulty"] >= difficulty_bins[i]) & (method_data["difficulty"] < difficulty_bins[i+1])
        if i == len(difficulty_bins)-2:  # Last bin includes upper bound
            mask = method_data["difficulty"] >= difficulty_bins[i]
        
        bin_data = method_data[mask]
        error_rate = bin_data["error"].mean() if len(bin_data) > 0 else 0
        bin_errors.append(error_rate)
    
    axes[0, 1].plot(bin_labels, bin_errors, marker='o', linewidth=2, label=f'{method.capitalize()}', color=color)

axes[0, 1].set_xlabel('Difficulty Level')
axes[0, 1].set_ylabel('Error Rate')
axes[0, 1].set_title('Error Rate by Difficulty Level')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)
axes[0, 1].set_ylim(0, 1)

# 3. Decision timeline for proposed method
proposed_decisions = df_proposed["decision"].values
decision_numeric = [1 if d == "fusion" else 0 for d in proposed_decisions]
examples = range(len(proposed_decisions))

axes[1, 0].plot(examples, decision_numeric, marker='o', linestyle='-', linewidth=2, markersize=4, color='green')
axes[1, 0].set_xlabel('Example Index')
axes[1, 0].set_ylabel('Decision (0=Fission, 1=Fusion)')
axes[1, 0].set_title('Decision Timeline (Proposed Method)')
axes[1, 0].set_yticks([0, 1])
axes[1, 0].set_yticklabels(['Fission', 'Fusion'])
axes[1, 0].grid(True, alpha=0.3)

# 4. Error occurrence vs decisions
proposed_errors = df_proposed["error"].astype(int)
axes[1, 1].scatter(examples, proposed_errors, c=[('orange' if d=='fusion' else 'blue') for d in proposed_decisions], 
                   alpha=0.7, s=50)
axes[1, 1].set_xlabel('Example Index')
axes[1, 1].set_ylabel('Error Occurred (0=No, 1=Yes)')
axes[1, 1].set_title('Errors vs Decisions (Orange=Fusion, Blue=Fission)')
axes[1, 1].set_yticks([0, 1])
axes[1, 1].set_yticklabels(['No Error', 'Error'])
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n📊 Visualization completed!")
print("🔍 Key insights:")
print("   • Baseline always chooses conservative fission mode")
print("   • Proposed method adaptively switches between modes")
print("   • DKW bounds provide statistical guarantees on error rates")

## 🎯 Interactive Exploration

Try modifying the controller parameters and rerunning the experiment to see how it affects behavior:

In [ ]:
# Interactive parameter exploration
def run_parameter_study(epsilon_target=0.10, delta=0.05, min_samples=100, hysteresis=0.05):
    """
    Run experiment with custom parameters.
    
    Args:
        epsilon_target: Target error threshold
        delta: Confidence parameter 
        min_samples: Minimum samples before decisions
        hysteresis: Hysteresis margin
    """
    print(f"🔧 Running experiment with custom parameters:")
    print(f"   epsilon_target: {epsilon_target}")
    print(f"   delta: {delta}")
    print(f"   min_samples: {min_samples}")
    print(f"   hysteresis: {hysteresis}")
    print()
    
    # Create custom controller
    custom_controller = DKWController(
        epsilon_target=epsilon_target,
        delta=delta, 
        min_samples=min_samples,
        hysteresis=hysteresis
    )
    
    # Run experiment with custom controller
    custom_results = {"baseline": [], "proposed": []}
    
    for example in experimental_data:
        error = np.random.random() < example["difficulty"]
        custom_controller.add_observation(float(error))
        decision = custom_controller.decide()
        
        custom_results["proposed"].append({
            "id": example["id"],
            "decision": decision,
            "error": error
        })
        
        custom_results["baseline"].append({
            "id": example["id"],
            "decision": "fission", 
            "error": error
        })
    
    # Analyze results
    fusion_count = sum(1 for r in custom_results["proposed"] if r["decision"] == "fusion")
    total_errors = sum(1 for r in custom_results["proposed"] if r["error"])
    
    print(f"📊 Results with custom parameters:")
    print(f"   Fusion decisions: {fusion_count}/{len(experimental_data)} ({fusion_count/len(experimental_data):.1%})")
    print(f"   Total errors: {total_errors}/{len(experimental_data)} ({total_errors/len(experimental_data):.1%})")
    
    return custom_results

# Example: Try more conservative settings
print("🔬 Experiment 1: More Conservative (lower target, higher hysteresis)")
conservative_results = run_parameter_study(epsilon_target=0.08, hysteresis=0.08)

print("\n" + "="*60 + "\n")

# Example: Try more aggressive settings  
print("🔬 Experiment 2: More Aggressive (higher target, lower hysteresis)")
aggressive_results = run_parameter_study(epsilon_target=0.15, hysteresis=0.02)

## 🎊 Conclusion

This notebook demonstrates a complete implementation of the **DKW Controller** that:

### ✅ **Key Features:**
- **Statistical Guarantees**: Uses DKW inequality for error bound confidence
- **Adaptive Decision Making**: Switches between fusion/fission modes based on observed data
- **Hysteresis Control**: Prevents oscillation between modes
- **Self-Contained**: No external file dependencies

### 🔍 **Key Insights:**
- The DKW controller provides theoretical guarantees on error rates
- Baseline approach is always conservative (100% fission decisions)
- Proposed approach adapts based on empirical error observations
- Parameter tuning allows control over conservative vs aggressive behavior

### 🚀 **Next Steps:**
- Experiment with different parameter combinations
- Try larger datasets to see convergence behavior
- Compare with other adaptive control strategies
- Apply to real-world decision making scenarios

---
**Original Script**: `method.py` from artifact `experiment_001`  
**Conversion**: Self-contained Jupyter notebook with inline data and interactive exploration